In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the dataset
df = pd.read_csv('database.csv')

df.head()

df.info()

df.keys()

df['classificacao'].unique()

factorized_mapping = dict(enumerate(df['classificacao'].unique()))
df['classificacao'] = pd.factorize(df['classificacao'])[0]
print(factorized_mapping)

features = df[['pressao_kpa','temperatura_c','luminosidade_lux','umidade_percentual']]
labels = df[['classificacao']]

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=0)

model = keras.Sequential([
keras.layers.Dense(units=1, input_shape=(4,))
])

model.compile(optimizer='Adam', loss='mean_squared_error')

model.fit(
features_train,labels_train,
epochs=6,
validation_data=(features_test, labels_test)
)

predictions = model.predict(features_test)
print(predictions)


original_predictions = [factorized_mapping[prediction] for prediction in predictions.argmax(axis=1)]
original_predictions

rmse = np.sqrt(((labels_test - predictions) ** 2).mean())
print("Root Mean Squared Error (RMSE):", rmse)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pressao_kpa         108 non-null    float64
 1   temperatura_c       108 non-null    int64  
 2   luminosidade_lux    108 non-null    int64  
 3   umidade_percentual  108 non-null    int64  
 4   classificacao       108 non-null    object 
dtypes: float64(1), int64(3), object(1)
memory usage: 4.3+ KB
{0: 'toleravel', 1: 'intoleravel', 2: 'otimo', 3: 'bom'}
Epoch 1/6
3/3 [==============================] - 1s 71ms/step - loss: 8472728.0000 - val_loss: 8931485.0000
Epoch 2/6
3/3 [==============================] - 0s 23ms/step - loss: 8423238.0000 - val_loss: 8878131.0000
Epoch 3/6
3/3 [==============================] - 0s 23ms/step - loss: 8372813.5000 - val_loss: 8825309.0000
Epoch 4/6
3/3 [==============================] - 0s 16ms/step - loss: 8323632.0000 - val_loss: 8772845.0000
Ep

In [ ]:
import tensorflow as tf

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("weather_model.tflite", "wb").write(tflite_model)

In [ ]:
import os
basic_model_size = os.path.getsize("weather_model.tflite")
print("Model is %d bytes" % basic_model_size)

"""# Encode the Model in an Arduino Header File"""

!echo "const unsigned char model[] = {" > /content/model.h
!cat weather_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

In [ ]:
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")